# 소비자심리지수 함수

## 0. 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

rc('font', family='MalgunGothic')
plt.rcParams['axes.unicode_minus'] = False

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/기상청/데이터')

In [4]:
df_final = pd.read_csv('./소비자심리지수.csv',encoding='CP949')

In [5]:
df_final.head()

,시도,2018. 01,2018. 02,2018. 03,2018. 04,2018. 05,2018. 06,2018. 07,2018. 08,2018. 09,2018. 10,2018. 11,2018. 12,2019. 01,2019. 02,2019. 03,2019. 04,2019. 05,2019. 06,2019. 07,2019. 08,2019. 09,2019. 10,2019. 11,2019. 12,2020. 01,2020. 02,2020. 03,2020. 04,2020. 05,2020. 06,2020. 07,2020. 08,2020. 09,2020. 10,2020. 11,2020. 12
0,강릉,102.500000,98.333333,101.333333,97.333333,101.833333,99.000000,97.666667,93.000000,86.166667,86.333333,82.166667,82.666667,86.666667,88.000000,87.500000,87.000000,84.166667,86.333333,86.166667,83.166667,87.333333,86.833333,88.500000,87.666667,89.666667,86.666667,70.000000,67.166667,75.000000,79.000000,79.000000,80.000000,71.333333,78.833333,83.833333,78.333333
1,강원,103.166667,100.000000,101.666667,98.333333,101.833333,98.500000,95.000000,91.500000,86.000000,87.333333,84.500000,84.500000,87.000000,88.500000,88.166667,89.833333,86.833333,87.166667,87.000000,84.666667,88.166667,88.166667,88.333333,88.833333,90.666667,88.000000,72.833333,70.000000,75.666667,78.166667,79.166667,80.666667,72.833333,81.166667,85.000000,78.833333
2,경기,102.333333,102.666667,101.333333,100.500000,102.166667,98.833333,95.500000,91.833333,90.666667,90.500000,87.000000,87.833333,89.500000,90.666667,90.000000,92.333333,89.833333,89.500000,88.500000,86.333333,89.333333,91.166667,93.666667,93.500000,95.500000,89.666667,75.666667,71.666667,76.000000,78.500000,81.833333,83.833333,75.833333,86.500000,92.166667,84.500000
3,경남,94.666667,92.833333,93.166667,93.000000,94.666667,91.500000,88.666667,87.500000,83.666667,84.166667,80.666667,81.333333,83.000000,83.833333,84.833333,87.500000,85.666667,84.000000,84.000000,81.500000,83.500000,85.000000,87.833333,87.500000,88.666667,85.333333,71.500000,63.833333,71.333333,72.500000,76.666667,79.666667,70.500000,79.500000,85.833333,80.500000
4,광주전남,104.000000,103.666667,102.000000,102.000000,104.000000,101.500000,97.166667,93.500000,92.166667,94.333333,92.833333,91.500000,93.166667,95.500000,94.166667,94.333333,92.833333,94.000000,91.833333,88.000000,93.333333,93.833333,95.500000,96.166667,97.166667,91.000000,76.500000,73.500000,76.833333,80.166667,77.833333,82.500000,74.500000,84.666667,89.833333,83.500000


## 1. 데이터 재구조화

In [6]:
df_melt = pd.melt(df_final, id_vars=['시도'])

In [7]:
df_melt['yymm'] = df_melt['variable'].apply(lambda x: x[:4] + x[6:8])

In [29]:
print(df_melt.shape)

(504, 4)


In [30]:
df_melt.head()

,시도,variable,value,yymm
0,강릉,2018. 01,102.500000,201801
1,강원,2018. 01,103.166667,201801
2,경기,2018. 01,102.333333,201801
3,경남,2018. 01,94.666667,201801
4,광주전남,2018. 01,104.000000,201801


In [9]:
df_melt['시도'].unique()

array(['강릉', '강원', '경기', '경남', '광주전남', '대구경북', '대전충남', '부산', '울산', '인천',
       '전북', '제주', '충북', '서울'], dtype=object)

## 2. 함수

* 강원 숫자 처리 필요

In [48]:
def score(data, col_name):
    # 데이터 전처리
    data['시도'] = data['aws_id'].map({108:'서울', 159:'부산', 143:'대구', 112:'인천', 156:'광주',
                                     133:'대전', 152:'울산', 119:'경기', 105:'강원', 131:'충북', 232:'충남',
                                     146:'전북', 168:'전남', 136:'경북', 155:'경남', 184:'제주'})
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['yymm'] = data['year'].map(str) + data['month'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))

    # score 계산
    data['score'] = np.nan
    for i in range(len(data)):
        for j in range(len(df_melt)):
            if (data.loc[i,'yymm'] == df_melt.loc[j,'yymm']) & (data.loc[i,'시도'] in df_melt.loc[j,'시도']):
                data.loc[i,'score'] = data.loc[i,col_name]*(df_melt.loc[j,'value']/100)
    return pd.DataFrame(data.groupby(['date'])['score'].mean()).reset_index()

## 3. 실행예시

In [14]:
example = pd.DataFrame({'date' : ['2018-01-01','2018-01-01','2018-01-01','2018-02-01','2018-12-31'],
                        'aws_id' : [108,159,143,112,156],
                        'hm_max' : [91,25,89,67,85]})

In [15]:
example

,date,aws_id,hm_max
0,2018-01-01,108,91
1,2018-01-01,159,25
2,2018-01-01,143,89
3,2018-02-01,112,67
4,2018-12-31,156,85


In [49]:
daily_score = score(example, 'hm_max')

In [50]:
daily_score

,date,score
0,2018-01-01,66.454444
1,2018-02-01,65.436667
2,2018-12-31,77.775000


In [46]:
# 확인
print((df_melt[(df_melt['yymm']=='201801')&(df_melt['시도']=='서울')].value/100)*91)
print((df_melt[(df_melt['yymm']=='201801')&(df_melt['시도']=='부산')].value/100)*25)
print((df_melt[(df_melt['yymm']=='201801')&(df_melt['시도']=='대구경북')].value/100)*89)

13    90.696667
Name: value, dtype: float64
7    23.375
Name: value, dtype: float64
5    85.291667
Name: value, dtype: float64


In [47]:
(sum([90.696667,23.375,85.291667])/3)

66.45444466666667